In [8]:
!pip install -q --user --upgrade --pre pythainlp 

# Workshop Notebook 1: Getting started with PyThaiNLP 😆


Updated: 31 October 2019

## Header


In [1]:
from typing import Set, List
from functools import reduce
import re

## 1. Word Tokenization

Word Tokenization is a process to determin word boundaries in a text or sentence.


Given a sentence, the tokenizer then read the sentence and return a list of words (i.e. tokens).

```python

    definition: Tokenizer(str) -> List[str]
    
    
    
    Tokenizer(str:"เธอคือ My Ambulance ของฉัน")  -> List["เธอ", "คือ", "My", "Ambulance", "ของ", "ฉัน"]

```



### Dictionary-based tokenizer


Dictionary-based tokenizer is an alogirithm the read through the sentence character by character.  If it found sequences of characters match with a vocabulary in the pre-defined dictionary, it maps sequences of characters as a token.
https://www.cs.ait.ac.th/~mdailey/papers/Choochart-Wordseg.pdf


```python

dictionary = Set["ฉัน", "ชอบ", "รถไฟ", "รถ", "รด", "น่ำ", "ต้น", "ไม้", "ต้นไม้", " "]


Dictionary_Tokenizer(dictionary:Set[str])

```




#### 1.1 Longest matching (LM)

Longest matching is an algorithm to split words from a sentence by considering logest vocab first.

In [2]:
dictionary = set(["ฉัน", "ชอบ", "รถไฟ", "รถ", "รด", "น่ำ", "ต้น", "ไม้", "ต้นไม้", " ", "ฟ้า"])

def search_longest(term, dictionary):
    term_length = len(term)
    max_length = 0
    for vocab in dictionary:
        if term in vocab:
            max_length = max(max_length, len(vocab))

    return max_length == term_length

def Dictionary_Tokenizer_LM_debug(sentence:str, dictionary: Set[str]):
    buffer = ""
    tokens = []
    for char in sentence:
        buffer += char
        print("buffer", buffer)
        if search_longest(buffer, dictionary) == True:
            print("select this token: {}".format(buffer))
            tokens.append(buffer)
            buffer = ""
            print("clear the buffer.")
            print("")
    return tokens

def Dictionary_Tokenizer_LM(sentence:str, dictionary: Set[str]):
    buffer = ""
    tokens = []
    for char in sentence:
        buffer += char
        if search_longest(buffer, dictionary) == True:
            tokens.append(buffer)
            buffer = ""
    return tokens

In [3]:
Dictionary_Tokenizer_LM_debug("ฉันชอบ รถไฟ", dictionary)

buffer ฉ
buffer ฉั
buffer ฉัน
select this token: ฉัน
clear the buffer.

buffer ช
buffer ชอ
buffer ชอบ
select this token: ชอบ
clear the buffer.

buffer  
select this token:  
clear the buffer.

buffer ร
buffer รถ
buffer รถไ
buffer รถไฟ
select this token: รถไฟ
clear the buffer.



['ฉัน', 'ชอบ', ' ', 'รถไฟ']

#### __Question 1:__ Create your own dictionary to tokenize the following sentences that can tokenize all the words.

In [4]:
test_sentences = [
    "กระทรวงคมนาคมและการสื่อสารกาตาร์ จัดงาน Qatar Information Technology Exhibition and Conference (QITCOM 2019)",
    "ณ กรุงโดฮา รัฐกาตาร์",
]

In [79]:
# Fill the vocabulary to dictionary_lm

dictionary_lm = set([
    " ",
    "Qatar",
    "Information",
    "กระทรวงคมนาคมและการสื่อสาร",
    "กาตาร์",
    # add more vocab
])

__Test:__

In [80]:
def test_Dictionary_Tokenizer_LM(dictionary_lm):
    
    tokens_list = [ Dictionary_Tokenizer_LM(sentence, dictionary_lm) for sentence in test_sentences]
    character_count_expect = sum([len(sentence) for sentence in test_sentences])
    character_count_actual = 0
    for tokens in tokens_list:
        character_count_actual += sum(map(lambda token : len(token),tokens))

    if(character_count_actual == character_count_expect):
        print("✅ Test succeed. 😁")
        
        print("\n tokens_list: ", tokens_list)
    else:
        print("Test failed. 😭\n")
        
        print("test_sentences", test_sentences)
        print("tokens_list", tokens_list)
        
        print('')
        print("character_count_actual != character_count_expect")
        print("{} != {}".format(character_count_actual, character_count_expect))

In [81]:
# Run this block to test the code
test_Dictionary_Tokenizer_LM(dictionary_lm)

Test failed. 😭

test_sentences ['กระทรวงคมนาคมและการสื่อสารกาตาร์ จัดงาน Qatar Information Technology Exhibition and Conference (QITCOM 2019)', 'ณ กรุงโดฮา รัฐกาตาร์']
tokens_list [['กระทรวงคมนาคมและการสื่อสาร', 'กาตาร์', ' '], []]

character_count_actual != character_count_expect
33 != 128


__Solution:__

In [8]:
dictionary_lm = set([
    "Qatar",
    "Information",
    "กระทรวงคมนาคมและการสื่อสาร",
    "กาตาร์",
    "ณ",
    "กรุงโดฮา",
    "รัฐ",
    " ",
    "จัดงาน",
    "(",
    ")",
    "QITCOM", "2019",
    "Qatar",
    "Information",
    "Technology",
    "Exhibition",
    "and",
    "Conference"
])

In [9]:
test_Dictionary_Tokenizer_LM(dictionary_lm)

✅ Test succeed. 😁

 tokens_list:  [['กระทรวงคมนาคมและการสื่อสาร', 'กาตาร์', ' ', 'จัดงาน', ' ', 'Qatar', ' ', 'Information', ' ', 'Technology', ' ', 'Exhibition', ' ', 'and', ' ', 'Conference', ' ', '(', 'QITCOM', ' ', '2019', ')'], ['ณ', ' ', 'กรุงโดฮา', ' ', 'รัฐ', 'กาตาร์']]


#### 1.2 Maximal matching (MM)


Unlike Longest Matching, Maximal matching is an algorithm to split words from a sentence in which it prefers minumum number of tokens to be splited.


```python

dictionary = set(["รถ", "รถไฟ", "ฟ้า", "ไฟฟ้า", "ใต้ดิน"])


sentence = "รถไฟฟ้าใต้ดิน"

Possible_segments(sentence) ->
["รถไฟ", "ฟ้า", "ใต้ดิน"]
["รถ", "ไฟฟ้า", "ใต้ดิน"]
["รถไฟฟ้า", "ใต้ดิน"]


selected_segment = ["รถไฟฟ้า", "ใต้ดิน"]

```


#### PyThaiNLP's Tokenizer (newmm)

In [10]:
from pythainlp.tokenize import word_tokenize

In [11]:
test_sentence = "กระทรวงคมนาคมและการสื่อสารกาตาร์ จัดงาน Qatar Information Technology Exhibition and Conference (QITCOM 2019)"


In [12]:
tokens = word_tokenize(test_sentence, engine="newmm")
print(tokens)

['กระทรวงคมนาคม', 'และ', 'การสื่อสาร', 'กาตาร์', ' ', 'จัดงาน', ' ', 'Qatar', ' ', 'Information', ' ', 'Technology', ' ', 'Exhibition', ' ', 'and', ' ', 'Conference', ' ', '(', 'QITCOM', ' ', '2019', ')']


__Try out:__

2.1 Try adding your own sentence.



In [82]:
# Example sentence
print(word_tokenize("ฉันอยุ๋ที่ สถาบันบัณฑิตพัฒนบริหารศาสตร์", engine="newmm"))

['ฉัน', 'อยุ๋', 'ที่', ' ', 'สถาบันบัณฑิตพัฒนบริหารศาสตร์']


In [83]:
# Enter you own setnence
print(word_tokenize(" ", engine="newmm"))

[' ']


2.2 Try adding your own sentence with misspelling.


In [84]:
# Example sentence with misspelling words
print(word_tokenize("ฉันอยุ๋ที่ สถาบันบัณฑิตพัฒนบยริหารศาสตร์", engine="newmm"))

['ฉัน', 'อยุ๋', 'ที่', ' ', 'สถาบัน', 'บัณฑิต', 'พัฒน', 'บย', 'ริ', 'หาร', 'ศาสตร์']


In [85]:
# Enter you own setnence
print(word_tokenize(" ", engine="newmm"))

[' ']


#### __Question 2:__ Add your own custom dictionary for `newmm` tokenizer to tokenize the into the following tokens:

```

"วันที่ 22 ต.ค. เอเอฟพีรายงานว่า สมเด็จพระจักรพรรดินารุฮิโตะ ทรงเข้าพระราชพิธีบรมราชาภิเษก เป็นสมเด็จพระจักรพรรดิแห่งญี่ปุ่นโดยสมบูรณ์แล้ววันนี้ ที่พระราชวังหลวงในกรุงโตเกียว",

```

Result with the default dictionary:

```
['วันที่', ' ', '22', ' ', 'ต.ค.', ' ', 'เอเอฟพี', 'รายงาน', 'ว่า', ' ', 'สมเด็จ', 'พระ', 'จักรพรรดิ', 'นา', 'รุ', 'ฮิ', 'โตะ', ' ', 'ทรง', 'เข้า', 'พระราชพิธี', 'บรมราชาภิเษก', ' ', 'เป็น', 'สมเด็จ', 'พระ', 'จักรพรรดิ', 'แห่ง', 'ญี่ปุ่น', 'โดย', 'สมบูรณ์', 'แล้ว', 'วันนี้', ' ', 'ที่', 'พระราชวัง', 'หลวง', 'ใน', 'กรุง', 'โตเกียว']
```

Expectation:
```
['วันที่', ' ', '22', ' ', 'ต.ค.', ' ', 'เอเอฟพี', 'รายงาน', 'ว่า', ' ', 'สมเด็จพระจักรพรรดิ', 'นารุฮิโตะ', ' ', 'ทรง', 'เข้า', 'พระราชพิธี', 'บรมราชาภิเษก', ' ', 'เป็น', 'สมเด็จพระจักรพรรดิ', 'แห่ง', 'ญี่ปุ่น', 'โดย', 'สมบูรณ์', 'แล้ว', 'วันนี้', ' ', 'ที่', 'พระราชวัง', 'หลวง', 'ใน', 'กรุงโตเกียว']
```

In [86]:
from pythainlp.tokenize.trie import Trie
from pythainlp.corpus import thai_words

In [87]:
text_from_news = """วันที่ 22 ต.ค. เอเอฟพีรายงานว่า สมเด็จพระจักรพรรดินารุฮิโตะ ทรงเข้าพระราชพิธีบรมราชาภิเษก เป็นสมเด็จพระจักรพรรดิแห่งญี่ปุ่นโดยสมบูรณ์แล้ววันนี้ ที่พระราชวังหลวงในกรุงโตเกียว"""

In [88]:
# Add vocab in this list
custom_vocab = [
    
    
]


__Test:__

In [89]:
def test_tokenize_japan_news(custom_vocab):
    expect = ['วันที่', ' ', '22', ' ', 'ต.ค.', ' ', 'เอเอฟพี', 'รายงาน', 'ว่า', ' ',
              'สมเด็จพระจักรพรรดิ', 'นารุฮิโตะ', ' ', 'ทรง', 'เข้า', 'พระราชพิธี', 'บรมราชาภิเษก',
              ' ', 'เป็น', 'สมเด็จพระจักรพรรดิ', 'แห่ง', 'ญี่ปุ่น', 'โดย', 'สมบูรณ์', 'แล้ว', 'วันนี้',
              ' ', 'ที่', 'พระราชวัง', 'หลวง', 'ใน', 'กรุงโตเกียว']
    
    custom_dict_trie = Trie( list(thai_words()) + custom_vocab)

    actual = word_tokenize(text_from_news, custom_dict=custom_dict_trie, engine="newmm")
    
    
   
    if actual == expect:
        print("✅ Test succeed. 😁")
    else:
        print("❌ Test failed. 😭")
        print("\nYour result    :\n\n", "|".join(actual))
        print("\nExtected result:\n\n", "|".join(expect))

In [90]:
test_tokenize_japan_news(custom_vocab)

❌ Test failed. 😭

Your result    :

 วันที่| |22| |ต.ค.| |เอเอฟพี|รายงาน|ว่า| |สมเด็จ|พระ|จักรพรรดิ|นา|รุ|ฮิ|โตะ| |ทรง|เข้า|พระราชพิธี|บรมราชาภิเษก| |เป็น|สมเด็จ|พระ|จักรพรรดิ|แห่ง|ญี่ปุ่น|โดย|สมบูรณ์|แล้ว|วันนี้| |ที่|พระราชวัง|หลวง|ใน|กรุง|โตเกียว

Extected result:

 วันที่| |22| |ต.ค.| |เอเอฟพี|รายงาน|ว่า| |สมเด็จพระจักรพรรดิ|นารุฮิโตะ| |ทรง|เข้า|พระราชพิธี|บรมราชาภิเษก| |เป็น|สมเด็จพระจักรพรรดิ|แห่ง|ญี่ปุ่น|โดย|สมบูรณ์|แล้ว|วันนี้| |ที่|พระราชวัง|หลวง|ใน|กรุงโตเกียว


__Solution:__

In [91]:
# Add vocab
custom_vocab = [
    "สมเด็จพระจักรพรรดิ",
    "กรุงโตเกียว",
    "นารุฮิโตะ"
]

test_tokenize_japan_news(custom_vocab)

✅ Test succeed. 😁


### Learning-based tokenizer


Tokenizer is a Machine Learning model and train on supervised daataset (labeled dataset).


For example, one tokenizer of PyThaiNLP (`attacut`) uses Convolutional-neural Network to read the whole text and then determind word boundaries.

![attacut](images/attacut.png)

#### attacut

In [94]:
!pip install -q attacut

In [95]:
test_sentence = "กระทรวงคมนาคมและการสื่อสารกาตาร์ จัดงาน Qatar Information Technology Exhibition and Conference (QITCOM 2019)"

tokens = word_tokenize(test_sentence, engine="attacut")
print(tokens)

['กระทรวงคมนาคม', 'และ', 'การสื่อสารกาตาร์', ' ', 'จัด', 'งาน', ' ', 'Qatar', ' ', 'Information', ' ', 'Technology', ' ', 'Exhibition', ' ', 'and', ' ', 'Conference', ' ', '(', 'QITCOM', ' ', '2019', ')']


In [96]:
test_sentence = "ฉันอยุ๋ที่ สถาบันบัณฑิตพัฒนบริหารศาสตร์"

tokens = word_tokenize(test_sentence, engine="attacut")
print(tokens)

['ฉัน', 'อยุ๋', 'ที่', ' ', 'สถาบันบัณฑิตพัฒนบริหารศาสตร์']


__Try out:__ Try adding your own sentence.



In [98]:
# Enter you own setnence
print(word_tokenize(" ", engine="attacut"))



[' ']


## 2. Part of speech and Named Entity Recognition Tagging


In [105]:
!pip install -q sklearn_crfsuite

In [106]:
from pythainlp.tag.named_entity import ThaiNameTagger

tagger = ThaiNameTagger()

#### Named Entitiy Regcognition (NER) Tags:

|       Tags       |      Examples                       |
|------------------|-------------------------------------|
        DATE       |   1 ตุลาคม 2012                      |
        EMAIL      |   hr@mycompany.com                  |    
        LAW        |  พรบ.คุ้มครองผู้บริโภค                   |
        LEN        |       80 กิโลเมตร                    |     
      LOCATION     |  กรุงเทพ, ประเทศจีน, เอเวอเรสต์        | 
        MONEY      |   2,190 ล้านบาท                      |
    ORGANIZATION   |  คณะอักษรศาสตร์ จุฬาลงกรณ์มหาวิทยาลัย     |
       PERCENT     |   95.34%, 10เปอร์เซนต์                |
       PERSON      |   อรรถพล ธำรงรัตนฤทธิ์                 |
        PHONE      |   +6611-123-1123                    |
         TIME      |      14:20 น, เวลาเที่ยงตรง           |
          URL      |     mycompany.com                   |
         ZIP       |     รหัสไปรณีย์ 21210                  |

In [107]:
sentence = "วันนี้ วันที่ 1 ตุลาคม ได้ไปงานเปิดบ้าน ที่มหาวิทยาลัยธรรมศาสตร์"

In [108]:
tagger.get_ner(sentence, pos=False)

[('วันนี้', 'B-DATE'),
 (' ', 'O'),
 ('วันที่', 'O'),
 (' ', 'O'),
 ('1', 'B-DATE'),
 (' ', 'I-DATE'),
 ('ตุลาคม', 'I-DATE'),
 (' ', 'O'),
 ('ได้', 'O'),
 ('ไป', 'O'),
 ('งาน', 'O'),
 ('เปิด', 'O'),
 ('บ้าน', 'O'),
 (' ', 'O'),
 ('ที่', 'O'),
 ('มหาวิทยาลัยธรรมศาสตร์', 'B-LOCATION')]

In [109]:
tagger.get_ner(sentence, pos=True)

[('วันนี้', 'NOUN', 'B-DATE'),
 (' ', 'PUNCT', 'O'),
 ('วันที่', 'NOUN', 'O'),
 (' ', 'PUNCT', 'O'),
 ('1', 'NUM', 'B-DATE'),
 (' ', 'PUNCT', 'I-DATE'),
 ('ตุลาคม', 'NOUN', 'I-DATE'),
 (' ', 'PUNCT', 'O'),
 ('ได้', 'AUX', 'O'),
 ('ไป', 'VERB', 'O'),
 ('งาน', 'NOUN', 'O'),
 ('เปิด', 'VERB', 'O'),
 ('บ้าน', 'NOUN', 'O'),
 (' ', 'PUNCT', 'O'),
 ('ที่', 'SCONJ', 'O'),
 ('มหาวิทยาลัยธรรมศาสตร์', 'NOUN', 'B-LOCATION')]

#### __Question 3:__ From the following setentences how many types of named-entity appear in the sentence


```text
เมื่อวันที่ ๓๑ ตุลาคม ๒๕๖๒ เวลา 13:00 น. ตามเวลาประเทศไทย
```





In [110]:
tagger.get_ner("เมื่อวันที่ ๓๑ ตุลาคม ๒๕๖๒ เวลา 13:00 น. ตามเวลาประเทศไทย", pos=False)

[('เมื่อ', 'O'),
 ('วันที่', 'O'),
 (' ', 'O'),
 ('๓๑', 'B-DATE'),
 (' ', 'I-DATE'),
 ('ตุลาคม', 'I-DATE'),
 (' ', 'I-DATE'),
 ('๒๕๖๒', 'I-DATE'),
 (' ', 'O'),
 ('เวลา', 'O'),
 (' ', 'O'),
 ('13', 'B-TIME'),
 (':', 'I-TIME'),
 ('00', 'I-TIME'),
 (' ', 'I-TIME'),
 ('น.', 'I-TIME'),
 (' ', 'O'),
 ('ตามเวลา', 'O'),
 ('ประเทศ', 'O'),
 ('ไทย', 'B-LOCATION')]

#### __Question 4:__ From the following setentences how many types of named-entity appear in the sentence

Reference: https://www.khaosod.co.th/around-the-world-news/news_2993136

```text
วันที่ 22 ต.ค. เอเอฟพีรายงานว่า สมเด็จพระจักรพรรดินารุฮิโตะ ทรงเข้าพระราชพิธีบรมราชาภิเษก
เป็นสมเด็จพระจักรพรรดิแห่งญี่ปุ่นโดยสมบูรณ์แล้ววันนี้ ที่พระราชวังหลวงในกรุงโตเกียว
```





In [111]:
# Try out
tagger.get_ner("วันที่ 22 ต.ค. เอเอฟพีรายงานว่า สมเด็จพระจักรพรรดินารุฮิโตะ ทรงเข้าพระราชพิธีบรมราชาภิเษก เป็นสมเด็จพระจักรพรรดิแห่งญี่ปุ่นโดยสมบูรณ์แล้ววันนี้ ที่พระราชวังหลวงในกรุงโตเกียว", pos=False)

[('วันที่', 'O'),
 (' ', 'O'),
 ('22', 'B-DATE'),
 (' ', 'I-DATE'),
 ('ต.ค.', 'I-DATE'),
 (' ', 'O'),
 ('เอเอฟพี', 'B-ORGANIZATION'),
 ('รายงาน', 'O'),
 ('ว่า', 'O'),
 (' ', 'O'),
 ('สมเด็จ', 'B-PERSON'),
 ('พระ', 'I-PERSON'),
 ('จักรพรรดิ', 'I-PERSON'),
 ('นา', 'I-PERSON'),
 ('รุ', 'I-PERSON'),
 ('ฮิ', 'I-PERSON'),
 ('โตะ', 'I-PERSON'),
 (' ', 'O'),
 ('ทรง', 'O'),
 ('เข้า', 'O'),
 ('พระราชพิธี', 'O'),
 ('บรมราชาภิเษก', 'O'),
 (' ', 'O'),
 ('เป็น', 'O'),
 ('สมเด็จ', 'O'),
 ('พระ', 'O'),
 ('จักรพรรดิ', 'O'),
 ('แห่ง', 'O'),
 ('ญี่ปุ่น', 'B-LOCATION'),
 ('โดย', 'O'),
 ('สมบูรณ์', 'O'),
 ('แล้ว', 'O'),
 ('วันนี้', 'B-DATE'),
 (' ', 'O'),
 ('ที่', 'O'),
 ('พระราชวัง', 'B-LOCATION'),
 ('หลวง', 'I-LOCATION'),
 ('ใน', 'O'),
 ('กรุง', 'B-LOCATION'),
 ('โตเกียว', 'I-LOCATION')]

#### __Question 5:__ From the following setentences how many types of named-entity appear in the sentence

Reference: [link](http://www.arts.chula.ac.th/ling/blog/tag/%E0%B8%AD%E0%B8%A3%E0%B8%A3%E0%B8%96%E0%B8%9E%E0%B8%A5-%E0%B8%98%E0%B8%B3%E0%B8%A3%E0%B8%87%E0%B8%A3%E0%B8%B1%E0%B8%95%E0%B8%99%E0%B8%A4%E0%B8%97%E0%B8%98%E0%B8%B4%E0%B9%8C/)

```text
คณะอักษรศาสตร์ จุฬาลงกรณ์มหาวิทยาลัย ขอเชิญชวนผู้สนใจเข้าร่วมฟังบรรยายพิเศษ
เรื่อง “การวิเคราะห์ความสัมพันธ์ภายในปริจเฉทแบบอัตโนมัติด้วยการจำแนกคำเชื่อม”
โดย ดร.อรรถพล ธำรงรัตนฤทธิ์

วันศุกร์ที่ 17 พฤศจิกายน 2560 เวลา 13.30-14.30 น.
เป็นต้นไป ณ ห้อง 401/5 อาคารมหาจักรีสิรินธร คณะอักษรศาสตร์ จุฬาลงกรณ์มหาวิทยาลัย

สอบถามรายละเอียดเพิ่มเติมได้ที่ 0-2218-4692
```





In [112]:
text = """คณะอักษรศาสตร์ จุฬาลงกรณ์มหาวิทยาลัย
ขอเชิญชวนผู้สนใจเข้าร่วมฟังบรรยายพิเศษ 
เรื่อง “การวิเคราะห์ความสัมพันธ์ภายในปริจเฉทแบบอัตโนมัติด้วยการจำแนกคำเชื่อม”
โดย ดร.อรรถพล ธำรงรัตนฤทธิ์
วันศุกร์ที่ 17 พฤศจิกายน 2560 เวลา 13.30-14.30 น. เป็นต้นไป ณ ห้อง 401/5 อาคารมหาจักรีสิรินธร คณะอักษรศาสตร์ จุฬาลงกรณ์มหาวิทยาลัย
สอบถามรายละเอียดเพิ่มเติมได้ที่ 0-2218-4692
"""

tagger.get_ner(text, pos=False)

[('คณะอักษรศาสตร์', 'B-ORGANIZATION'),
 (' ', 'O'),
 ('จุฬาลงกรณ์', 'B-ORGANIZATION'),
 ('มหาวิทยาลัย', 'I-ORGANIZATION'),
 ('\n', 'O'),
 ('ขอ', 'O'),
 ('เชิญชวน', 'O'),
 ('ผู้', 'O'),
 ('สนใจ', 'O'),
 ('เข้าร่วม', 'O'),
 ('ฟัง', 'O'),
 ('บรรยาย', 'O'),
 ('พิเศษ', 'O'),
 (' ', 'O'),
 ('\n', 'O'),
 ('เรื่อง', 'O'),
 (' ', 'O'),
 ('“', 'O'),
 ('การ', 'O'),
 ('วิเคราะห์', 'O'),
 ('ความสัมพันธ์', 'O'),
 ('ภายใน', 'O'),
 ('ปริ', 'O'),
 ('จ', 'O'),
 ('เฉท', 'O'),
 ('แบบ', 'O'),
 ('อัตโนมัติ', 'O'),
 ('ด้วย', 'O'),
 ('การจำแนก', 'O'),
 ('คำเชื่อม', 'O'),
 ('”', 'O'),
 ('\n', 'O'),
 ('โดย', 'O'),
 (' ', 'O'),
 ('ดร.', 'B-PERSON'),
 ('อรรถ', 'I-PERSON'),
 ('พล', 'I-PERSON'),
 (' ', 'I-PERSON'),
 ('ธำรง', 'I-PERSON'),
 ('รัตน', 'I-PERSON'),
 ('ฤทธิ์', 'I-PERSON'),
 ('\n', 'I-PERSON'),
 ('วัน', 'I-PERSON'),
 ('ศุกร์', 'I-PERSON'),
 ('ที่', 'O'),
 (' ', 'O'),
 ('17', 'B-DATE'),
 (' ', 'I-DATE'),
 ('พฤศจิกายน', 'I-DATE'),
 (' ', 'I-DATE'),
 ('2560', 'I-DATE'),
 (' ', 'O'),
 ('เวลา', 'O'),
 (' ', 'O

#### Part of Speech (POS) Tags:


Reference: [PUD Tags](https://universaldependencies.org/u/pos/all.html)




|  Abbreviation |      Part-of-Speech tag    |            Examples             |       
|---------------|----------------------------|---------------------------------|
| ADJ           |  Adjective                 |    ใหม่, พิเศษ , ก่อน, มาก, สูง     |   
| ADP           |  Adposition                |   แม้, ว่า, เมื่อ, ของ, สำหรับ       |   
| ADV           |  Adverb                    |   ก่อน, ก็, เล็กน้อย, เลย, สุด       |   
| AUX           |  Auxiliary                 |   เป็น, ใช่, คือ, คล้าย             |   
| CCONJ         |  Coordinating conjunction  |   แต่, และ, หรือ                  |        
| DET           |  Determiner                |   นี้, นั้น, ทั้ง, เพียง, (หนึ่ง)คน      |   
| INTJ          |  Interjection              |   อุ้ย, โอ้ย                       |   
| NOUN          |  Noun                      |   กำมือ, พวก, สนาม, กีฬา, บัญชี     |   
| NUM           |  Numeral                   |   5,000, 103.7, 2004, หนึ่ง, ร้อย  |   
| PART          |  Particle                  |   มา ขึ้น ไม่ ได้ เข้า               |      
| PRON          |  Pronoun                   |   เรา, เขา, ตัวเอง, ใคร, เธอ     |   
| PROPN         |  Proper noun               |   โอบามา, แคปิตอลฮิล, จีโอพี, ไมเคิล |   
| PUNCT         |  Punctuation               |   (, ), ", ', :                 |    
| SCONJ         |  Subordinating conjunction |    หาก, เพ่ราะว่า, ถ้า             |   
| VERB          |  Verb                      |   เปิด, ให้, ใช้, เผชิญ, อ่าน        |


In [35]:
from pythainlp.tag import pos_tag

In [36]:

sentence = "ฉันไปเดินในสวนสาธารณะ"
tokens = word_tokenize(sentence, keep_whitespace=False)
pos_tag(tokens, corpus="pud", engine="perceptron")


[('ฉัน', 'PRON'),
 ('ไป', 'VERB'),
 ('เดิน', 'VERB'),
 ('ใน', 'ADP'),
 ('สวนสาธารณะ', 'NOUN')]


```
Explaination:

PRON = Pronoun 

VERB = Verb

ADP = Adposition

NOUN = Noun

```

In [37]:

sentence = "ฉันไปเดินในสวนสาธารณะ"
tokens = word_tokenize(sentence, keep_whitespace=False)
pos_tag(tokens, corpus="orchid", engine="perceptron")


[('ฉัน', 'PPRS'),
 ('ไป', 'VACT'),
 ('เดิน', 'VACT'),
 ('ใน', 'RPRE'),
 ('สวนสาธารณะ', 'NCMN')]

```
Explaination:

PPRS = Personal pronoun 

VACT = Active verb

RPRE = Preposition

NCMN = Common noun
```

#### __Question 6:__ From the following setentences what are the POS tags (based on UD)


```text
หมา และ แมว กำลังกิน อาหาร
```


Hint: Here is the list of POS tags of this sentence.

- NOUN = Noun
- CCONJ = Coordinating Conjunction
- VERB = Active Verb



In [38]:
# Run this block to see the result

sentence = "หมาและแมวกำลังกินอาหาร"
tokens = word_tokenize(sentence, keep_whitespace=False)
pos_tag(tokens, corpus="ud", engine="perceptron")


[('หมา', 'NOUN'),
 ('และ', 'CCONJ'),
 ('แมว', 'NOUN'),
 ('กำลังกิน', 'VERB'),
 ('อาหาร', 'NOUN')]

#### __Question 7:__ From the following setentences what are the POS tags (based on Orchid)


```text
หมา และ แมว กำลังกิน อาหาร
```

Hint: Here is the list of POS tags of this sentence.

- NCMN = Common Noun
- JCRG = Coordinating Conjunction
- VACT = Active Verb

In [39]:
# Run this block to see the result

sentence = "หมาและแมวกำลังกินอาหาร"
tokens = word_tokenize(sentence, keep_whitespace=False)
pos_tag(tokens, corpus="orchid", engine="perceptron")

[('หมา', 'NCMN'),
 ('และ', 'JCRG'),
 ('แมว', 'NCMN'),
 ('กำลังกิน', 'VACT'),
 ('อาหาร', 'NCMN')]

## 3. Spell checking




### 3.1 PyThaiNLP's spell checker

In [40]:
from pythainlp.spell import correct

In [41]:
mispelled_words = [
    "โรงพยาบาน",
    "สวัสดิ",
    "ประธาราธิปดี",
    "สัปปะรด",
    "สังเกตุ",
    "เหตการณ์",
    "อนุญาติ",
    "ฝักไฝ่",
    "นายกรัญมนตี"
]

In [42]:
for word in mispelled_words:
    print("{} -> {}".format(word, correct(word)))
    print("")

โรงพยาบาน -> โรงพยาบาล

สวัสดิ -> สวัสดี

ประธาราธิปดี -> ประธานาธิบดี

สัปปะรด -> สับปะรด

สังเกตุ -> สังเกต

เหตการณ์ -> เหตุการณ์

อนุญาติ -> อนุญาต

ฝักไฝ่ -> ฝักใฝ่

นายกรัญมนตี -> นายกรัฐมนตรี



__Try out:__ Put any mispelling words and correct them.

In [ ]:
correct("")

## 4. Utility functions

### 4.1 Thai digits and currency Conversion


In [114]:
from pythainlp.util import (
    thai_digit_to_arabic_digit,
    arabic_digit_to_thai_digit,
    bahttext,
    digit_to_text,
    thaiword_to_num)


In [45]:
thai_digit_to_arabic_digit("เมื่อวันที่ ๓๑ ตุลาคม ๒๕๖๒ เวลา ๑๓:๐๐ น. ตามเวลาประเทศไทย")

'เมื่อวันที่ 31 ตุลาคม 2562 เวลา 13:00 น. ตามเวลาประเทศไทย'

In [46]:
arabic_digit_to_thai_digit("เมื่อวันที่ 31 ตุลาคม2562 เวลา 13:00 น. ตามเวลาประเทศไทย")

'เมื่อวันที่ ๓๑ ตุลาคม๒๕๖๒ เวลา ๑๓:๐๐ น. ตามเวลาประเทศไทย'

In [47]:
bahttext(1234.24)

'หนึ่งพันสองร้อยสามสิบสี่บาทยี่สิบสี่สตางค์'

In [48]:
bahttext(21)

'ยี่สิบเอ็ดบาทถ้วน'

In [49]:
bahttext(240000000000)

'สองแสนสี่หมื่นล้านบาทถ้วน'

In [50]:
thaiword_to_num("หนึ่งร้อย")

100

In [51]:
digit_to_text("๓ ร้อยล้าน")

'สาม ร้อยล้าน'

In [52]:
digit_to_text("๓๑")

'สามหนึ่ง'

In [53]:
thaiword_to_num("หนึ่งพันหนึ่ง")

1001

In [54]:
thaiword_to_num("หนึ่งล้านหกสิบเอ็ด")

1000061

In [55]:
thaiword_to_num("พันล้าน")

1000000000

#### __Question 8 :__ Given a text representing an amont money, convert into number.

```
เพื่อเพิ่มมูลค่าการค้าซึ่งปัจจุบันมีประมาณ ๑๒,๕๐๐ ล้านดอลลาร์สหรัฐ ให้ทวีขึ้นเป็น ๓ หมื่นล้าน
```
->
```
เพื่อเพิ่มมูลค่าการค้าซึ่งปัจจุบันมีประมาณ 12,500 ล้านดอลลาร์สหรัฐ ให้ทวีขึ้นเป็น 3 หมื่นล้าน

```


In [56]:
text_with_thai_digits = "เพื่อเพิ่มมูลค่าการค้าซึ่งปัจจุบันมีประมาณ ๑๒,๕๐๐ ล้านดอลลาร์สหรัฐ ให้ทวีขึ้นเป็น ๓ หมื่นล้าน"

In [57]:
def convert(text):
    splits = text.split(" ")
    
    for index, split in enumerate(splits):
        
        if re.search(r"[๐-๙]", split):
            print("\nselcted split: ", split)
            ## Modify the following line to convert from thai digits to arabic

            splits[index] = split
            
            ##--------------------- ##
            print("convert to: ", splits[index])
    
    return " ".join(splits)
    

In [58]:
convert(text_with_thai_digits)


selcted split:  ๑๒,๕๐๐
convert to:  ๑๒,๕๐๐

selcted split:  ๓
convert to:  ๓


'เพื่อเพิ่มมูลค่าการค้าซึ่งปัจจุบันมีประมาณ ๑๒,๕๐๐ ล้านดอลลาร์สหรัฐ ให้ทวีขึ้นเป็น ๓ หมื่นล้าน'

__Test:__ Given a list of sentences, please return only Thai sentences.

In [59]:
def test_convert_thai_digits(convert):
    expect = "เพื่อเพิ่มมูลค่าการค้าซึ่งปัจจุบันมีประมาณ 12,500 ล้านดอลลาร์สหรัฐ ให้ทวีขึ้นเป็น 3 หมื่นล้าน"
    actual = convert(text_with_thai_digits)

    if actual == expect:
        print("✅ Test succeed. 😁")
    else:
        print("❌ Test failed. 😭")
        print("The actual results:", actual)

test_convert_thai_digits(convert)


selcted split:  ๑๒,๕๐๐
convert to:  ๑๒,๕๐๐

selcted split:  ๓
convert to:  ๓
❌ Test failed. 😭
The actual results: เพื่อเพิ่มมูลค่าการค้าซึ่งปัจจุบันมีประมาณ ๑๒,๕๐๐ ล้านดอลลาร์สหรัฐ ให้ทวีขึ้นเป็น ๓ หมื่นล้าน


__Solution:__

In [60]:
def convert(text):
    splits = text.split(" ")
    
    for index, split in enumerate(splits):
        
        if re.search(r"[๐-๙]", split):
            print("\nselcted split: ", split)
            ## Write the code to convert

            splits[index] = thai_digit_to_arabic_digit(split)
            
            
            print("convert to: ", splits[index])

    return " ".join(splits)
    

In [61]:
test_convert_thai_digits(convert)


selcted split:  ๑๒,๕๐๐
convert to:  12,500

selcted split:  ๓
convert to:  3
✅ Test succeed. 😁


### 4.2 Thai Word count

In [62]:
from pythainlp.util import countthai



Count percentage of Thai chacters in a text.

```python
 countthai(text:str) -> percentage:float
```



In [63]:
countthai("Hello world.")

0.0

In [64]:
countthai("สวัสดี ฉันชอบนั่งรถไฟ")

100.0

In [65]:
countthai("สวัสดี Jane Doe")

46.15384615384615

#### __Question 9:__ Given a list of sentences, please return only Thai sentences.

In [66]:
en_th_sentences = [
    "มันจะมีอะไรที่ทำให้ ผิดพลาดได้ล่ะ?",
    "สวัสดี... ...ดิ๊ก",
    "# Just to get a glimpse beyond this illusion #",
    "# I was soaring ever higher #",
    "# but I flew too high #",
    "    ใช่",
    "ใช่ เขาและแฟรงค์ และ แคส ถ้าขวดเหล้าเขา อยู่ในกระเป๋า",
    "มันก็ดีที่เราได้คราวลีย์ มาอยู่ฝั่งเราถูกมั้ย?",
    "คุณคราวลีย์ เรามีเรื่องต้องคุยกันเยอะเลย",
    "เชิญนั่ง",
    "== sync, corrected by elderman ==",
    "# though my eyes could see, I still was a blind man #",
    "# though my mind could think, I still was a madman #",
    "ได้เห็นว่า พวกมันทั้งหมดมาตามล่าเขา",
    "# I hear the voices when I'm dreaming #",
]

In [67]:
def test_thai_sentence(sentence):

    ## Write down the code, to return value True if the sentence is in Thai language
    
    
    
    ##
    return False

__Test:__

In [68]:
th_sentences = [
    "มันจะมีอะไรที่ทำให้ ผิดพลาดได้ล่ะ?",
    "สวัสดี... ...ดิ๊ก",
    "    ใช่",
    "ใช่ เขาและแฟรงค์ และ แคส ถ้าขวดเหล้าเขา อยู่ในกระเป๋า",
    "มันก็ดีที่เราได้คราวลีย์ มาอยู่ฝั่งเราถูกมั้ย?",
    "คุณคราวลีย์ เรามีเรื่องต้องคุยกันเยอะเลย",
    "เชิญนั่ง",
    "ได้เห็นว่า พวกมันทั้งหมดมาตามล่าเขา",
]

In [69]:
actual = list(filter(test_thai_sentence, en_th_sentences))
expect = th_sentences

if actual == expect:
    print("✅ Test succeed. 😁")
else:
    print("❌ Test failed. 😭")
    print("The actual results:", actual)
    print("\nThe expected sentences to be returned:\n")
    for i, sentence in enumerate(th_sentences):
        print(i+1, sentence)

❌ Test failed. 😭
The actual results: []

The expected sentences to be returned:

1 มันจะมีอะไรที่ทำให้ ผิดพลาดได้ล่ะ?
2 สวัสดี... ...ดิ๊ก
3     ใช่
4 ใช่ เขาและแฟรงค์ และ แคส ถ้าขวดเหล้าเขา อยู่ในกระเป๋า
5 มันก็ดีที่เราได้คราวลีย์ มาอยู่ฝั่งเราถูกมั้ย?
6 คุณคราวลีย์ เรามีเรื่องต้องคุยกันเยอะเลย
7 เชิญนั่ง
8 ได้เห็นว่า พวกมันทั้งหมดมาตามล่าเขา


__Solution:__

In [70]:
def test_thai_sentence(sentence):

    ## Write down the code, to return value True if the sentence is in Thai language
    
    if countthai(sentence)  >= 90.0:
        return True
    
    ##
    return False

In [71]:
actual = list(filter(test_thai_sentence, en_th_sentences))
expect = th_sentences

if actual == expect:
    print("✅ Test succeed. 😁")
else:
    print("❌ Test failed. 😭")
    print("The actual results:", actual)

✅ Test succeed. 😁


### 4.3 Data and time

This function uses Thai names and Thai Buddhist Era for these directives:

- __%a__ - abbreviated weekday name (e.g. “จ”, “อ”, “พ”, “พฤ”, “ศ”, “ส”, “อา”)

- __%A__ - full weekday name (e.g.“วันจันทร์”, “วันอังคาร”, “วันเสาร์”, “วันอาทิตย์”)

- __%b__ - abbreviated month name (e.g.“ม.ค.”,”ก.พ.”,”มี.ค.”,”เม.ย.”,”พ.ค.”,”มิ.ย.”, “ธ.ค.”)

- __%B__ - full month name (e.g. “มกราคม”, “กุมภาพันธ์”, “พฤศจิกายน”, “ธันวาคม”,)

- __%y__ - year without century (e.g. “56”, “10”)

- __%Y__ - year with century (e.g. “2556”, “2410”)

- __%c__ - date and time representation (e.g. “พ 6 ต.ค. 01:40:00 2519”)

- __%v__ - short date representation (e.g. ” 6-ม.ค.-2562”, “27-ก.พ.-2555”)

- __%d__ - day (e.g. "01", "07", 10", "31")

- __%-d__ - day with no zero padding (e.g. "1", "7",10", "31")
 
- __%H__  - hour (e.g. "01", "06", "23")

- __%-H__ - hour with no zero padding (e.g. "1", "6", "23"))

- __%M__  - minute (e.g. "1", "2", "11", "12")

- __%S__  - second (e.g. "1", "2", "11", "12")

In [72]:
from datetime import datetime
from pythainlp.util import thai_strftime


In [73]:
# Print the current date in Thai format

thai_strftime(datetime.now(), "%d %B %Y")

'01 พฤศจิกายน 2562'

#### __Question 10:__ Given a date time object, return the datetime string in the following format


```
วันศุกร์ ที่ 1 พฤศจิกายน ปี พ.ศ. 2562 เวลา 11 นาฬิกา 30 นาที 10 วินาที
```

In [74]:
datetime_object_workshop_day = datetime(year=2019, month=11, day=1,hour=11,minute=30,second=10)

def print_datetime_thai(datetime_object):
    # Write down the format string.
    fmt = ""
    return thai_strftime(datetime_object, fmt)

__Test:__

In [75]:
def test_print_datetime_thai(fn):
    expect = "วันศุกร์ ที่ 1 พฤศจิกายน พ.ศ. 2562 เวลา 11 นาฬิกา 30 นาที 10 วินาที"
    actual = fn(datetime_object_workshop_day)

    if actual == expect:
        print("✅ Test succeed. 😁\n")
        print("Your Result:",expect)
    else:
        print("❌ Test failed. 😭")
        print("\nYour result    :", actual)
        print("\nExpected result:", expect)

test_print_datetime_thai(print_datetime_thai)

❌ Test failed. 😭

Your result    : 

Expected result: วันศุกร์ ที่ 1 พฤศจิกายน พ.ศ. 2562 เวลา 11 นาฬิกา 30 นาที 10 วินาที


__Solution:__

In [76]:
def print_datetime_thai(datetime_object):
    # Write down the format string.
    fmt = "%A ที่ %-d %B พ.ศ. %Y เวลา %H นาฬิกา %M นาที %S วินาที"
    return thai_strftime(datetime_object, fmt)

test_print_datetime_thai(print_datetime_thai)

✅ Test succeed. 😁

Your Result: วันศุกร์ ที่ 1 พฤศจิกายน พ.ศ. 2562 เวลา 11 นาฬิกา 30 นาที 10 วินาที


__Without Thai strftime__

In [78]:
datetime_object_workshop_day.strftime("%A ที่ %-d %B พ.ศ. %Y เวลา %H นาฬิกา %M นาที %S วินาที")

'Friday ที่ 1 November พ.ศ. 2019 เวลา 11 นาฬิกา 30 นาที 10 วินาที'